# Imports 

In [7]:
## Essential Imports: 
import os
import numpy as np
import qp
import tables_io
from pathlib import Path 
from pzflow.examples import get_galaxy_data
import ceci

## RAIL-Specific Imports: 
import rail

# old : from rail.creation.degradation import LSSTErrorModel, InvRedshiftIncompleteness


from rail.creation.degradation.lsst_error_model import LSSTErrorModel
from rail.creation.degradation.spectroscopic_degraders import InvRedshiftIncompleteness

import rail.creation 
import rail.creation.engines
from rail.creation.engines.flowEngine import FlowModeler, FlowCreator, FlowPosterior
from rail.core.data import TableHandle
from rail.core.stage import RailStage
from rail.core.utilStages import ColumnMapper, TableConverter

# old : from rail.estimation.algos.flexzboost import Inform_FZBoost, FZBoost

from rail.estimation.algos.train_z import TrainZEstimator, TrainZInformer
from rail.estimation.algos.cmnn import Inform_CMNNPDF, CMNNPDF
from rail.estimation.algos.gpz import GPzInformer, GPzEstimator 
from rail.estimation.algos.pzflow_nf import PZFlowInformer, PZFlowEstimator 
from rail.estimation.algos.flexzboost import FlexZBoostInformer, FlexZBoostEstimator  


from rail.evaluation.evaluator import Evaluator


## Data Storage: 
DS = RailStage.data_store
DS.__class__.allow_overwrite = True


### CMNN, PZFlow, FlexZBoost, GPZ, trainz for control

In [8]:
help(rail.estimation.algos)

Help on package rail.estimation.algos in rail.estimation:

NAME
    rail.estimation.algos

PACKAGE CONTENTS
    _gpz_util
    bpz_lite
    cmnn
    delightPZ
    delight_version (package)
    equal_count
    flexzboost
    gpz
    naive_stack
    point_est_hist
    pzflow_nf
    random_gauss
    train_z
    uniform_binning
    var_inf

FILE
    (built-in)




In [9]:
# help(rail.creation.engines)

In [10]:
#from rail.stages import *
#rail.stages.import_and_attach_all()
#for val in RailStage.pipeline_stages.values():
#    print(val[0])

# Model

In [11]:
def makeModel():
    #path to access the data 
    DATA_DIR = Path().resolve() / "data"
    DATA_DIR.mkdir(exist_ok=True)

    catalog_file = DATA_DIR / "base_catalog.pq"

    bands = ['u','g','r','i','z','y']
    band_dict = {band:f'mag_{band}_lsst' for band in bands}
    
    #array of galaxies w/ 7 attributes for each: redshift & ugrizy
    catalog = get_galaxy_data().rename(band_dict, axis=1) 

    #turns array into a table 
    tables_io.write(catalog, str(catalog_file.with_suffix("")), catalog_file.suffix[1:])

    catalog_file = str(catalog_file)
    flow_file = str(DATA_DIR / "trained_flow.pkl")

    print(flow_file)

    #we set up the stage 
    flow_modeler_params = {
        "name": "flow_modeler",
        "input": catalog_file,
        "model": flow_file,
        "seed": 0,
        "phys_cols": {"redshift": [0, 3]},
        "phot_cols": {
            "mag_u_lsst": [17, 35],
            "mag_g_lsst": [16, 32],
            "mag_r_lsst": [15, 30],
            "mag_i_lsst": [15, 30],
            "mag_z_lsst": [14, 29],
            "mag_y_lsst": [14, 28],
        },
        "calc_colors": {"ref_column_name": "mag_i_lsst"},
    }
    flow_modeler = FlowModeler.make_stage(**flow_modeler_params)
    # flow_modeler.fit_model()
    return flow_modeler, flow_file ##.get_handle("model")

In [12]:
modelData, flow_file = makeModel() 

/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/data/trained_flow.pkl


# Make Training Set and Test Set 

In [13]:
def trainSet(ntrain, seed):
    data = FlowCreator.make_stage(
            name = 'train_set',
            model = flow_file,
            n_samples = ntrain,
            seed = seed 
    )
    return data 

In [14]:
def testSet(ntest, seed):
    data = FlowCreator.make_stage(
            name = 'test_set',
            model = flow_file,
            n_samples = ntest,
            seed = seed 
    )
    return data #.sample(ntest, seed)

# Degraders

## Inverse Redshift Incompleteness

In [15]:
def invRedshift(pivot = 1.0):
    assert type(pivot) == float 
    degr = InvRedshiftIncompleteness.make_stage(
        name = 'inv_redshift',
        pivot_redshift = pivot
    )
    return degr 

pivot_ls = [1.0, 1.4]

In [16]:
# ## Choose pivot z's for inverse redshift incompleteness 

# ## seed1 and ndata should be the same as  seed1 and ntrain used to call bigF!! 
# ## Otherwise this might not be representative of the real data 

# def choosePivots(seed1, ndata):
#     nums = trainSet(ndata, seed1)
#     data = nums.sample(ndata, seed1)
#     data_pq = col_remap(data)
#     data_table = table_conv(data_pq)
#     table = tables_io.convertObj(data_table.data, tables_io.types.PD_DATAFRAME)
#     return np.asarray(table['redshift'])

# percentiles = np.arange(10, 100, 10)
# pivots = [] 

# for i in percentiles:
#     pivot = np.percentile(choosePivots(17, 100000), i) 
#     pivots.append(pivot)

In [17]:
# print(pivots)

## LSST Error 

In [18]:
bands = ['u','g','r','i','z','y']
band_dict = {band:f'mag_{band}_lsst' for band in bands}

def lsstError(dict, seed): #tvis = 1, nYrObs = 1, airmass = 1, extendedSource = 1, sigmaSys = 1, magLim = 1, ndFlag = 1, A_min = 1, A_max = 1):
    deg = LSSTErrorModel.make_stage(
        name='lsst_error',
        renameDict= dict, 
        ndFlag=np.nan,
        seed=seed,
    )
    return deg 

## Quantity Cuts 

In [19]:
## write a dictionary with the different bands and magnitudes you want

def quantCuts(band, mag):
    quantity_cut = QuantityCut.make_stage(
        name='quantity_cut',    
        cuts={'mag_i_lsst': 25.0},
    )

In [20]:
qcuts_dict = {'mag_u_lsst': [...], 
              'mag_g_lsst': [...], 
              'mag_r_lsst': [...], 
              'mag_i_lsst': [...], 
              'mag_z_lsst': [...], 
              'mag_y_lsst': [...] }

## Survey-Based Degraders

In [21]:
from rail.creation.degradation.spectroscopic_selections import *

def specSelectBOSS(ntrain):
    degr = SpecSelection_BOSS.make_stage(
        name = 'specselection_boss',
        N_tot = ntrain
    )
    return degr 

def specSelectDEEP2(ntrain):
    degr = SpecSelection_DEEP2.make_stage(
        name = 'specselection_deep2',
        N_tot = ntrain
    )
    return degr 

def specSelectGAMA(ntrain):
    degr = SpecSelection_GAMA.make_stage(
        name = 'specselection_gama',
        N_tot = ntrain
    )
    return degr 

def specSelectHSC(ntrain):
    degr = SpecSelection_HSC.make_stage(
        name = 'specselection_HSC',
        N_tot = ntrain
    )
    return degr 

def specSelectVVDSf02(ntrain):
    degr = SpecSelection_VVDSf02.make_stage(
        name = 'specselection_VVDSf02',
        N_tot = ntrain
    )
    return degr 

def specSelectzCOSMOS(ntrain):
    degr = SpecSelection_zCOSMOS.make_stage(
        name = 'specselection_zCOSMOS',
        N_tot = ntrain
    )
    return degr 

In [22]:
spec_dict = {'BOSS': specSelectBOSS, 
             'DEEP2': specSelectDEEP2, 
             'GAMA': specSelectGAMA,
             'HSC': specSelectHSC, 
             'VVDSf02': specSelectVVDSf02, 
             'zCOSMOS': specSelectzCOSMOS } 

## Posts 

In [23]:
def getPosts(data, model, grid):
    posts = FlowPosterior.make_stage(
        name='get_posts'+str(data), 
        column='redshift',
        grid = grid,
        model = model,
        data = data
    )
    return posts #posts.get_posterior(data, column = 'redshift')

In [24]:
def makeGrid(zmin, zmax, nbins):
    import numpy as np
    grid = np.linspace(zmin, zmax, nbins + 1)
    return grid 

In [25]:
grid = makeGrid(0, 2.5, 100)

Only run if you need output_orig_train_posts

In [26]:
# flow_post_orig_train = FlowPosterior.make_stage(name='orig_train_posts', 
#                                              column='redshift',
#                                              grid = np.linspace(0, 2.5, 101),
#                                              model=flow_file,
#                                              data = orig_train)

# orig_train_pdfs = flow_post_orig_train.get_posterior(orig_train, column='redshift')

Only run if you need output_deg_train_posts ** rerun this cell!! 

In [27]:
# flow_post_deg_train = FlowPosterior.make_stage(name='deg_train_posts', 
#                                              column='redshift',
#                                              grid = np.linspace(0, 2.5, 101),
#                                              model=flow_file,
#                                              err_samples = 0,
#                                              data = deg_train)



# deg_train_pdfs = flow_post_deg_train.get_posterior(deg_train, column='redshift')

Only run if you need output_orig_test_posts

In [28]:
# flow_post_orig_test = FlowPosterior.make_stage(name='orig_test_posts', 
#                                              column='redshift',
#                                              grid = np.linspace(0, 2.5, 101),
#                                              model=flow_file,
#                                              data = orig_test)

# orig_test_pdfs = flow_post_orig_test.get_posterior(orig_test, column='redshift')

Only run if you need output_deg_test_posts

In [29]:
# flow_post_deg_test = FlowPosterior.make_stage(name='deg_test_posts', 
#                                              column='redshift',
#                                              grid = np.linspace(0, 2.5, 101),
#                                              model=flow_file,
#                                              data = deg_test)

# deg_test_pdfs = flow_post_deg_test.get_posterior(deg_test, column='redshift')

# Make tables

In [30]:
bands = ['u','g','r','i','z','y']
band_dict_err = {f'mag_{band}_lsst_err':f'mag_err_{band}_lsst' for band in bands}

def colRemapper(dict):
    col_remap = ColumnMapper.make_stage(
    name='col_remapper', 
    columns=dict,
    )
    return col_remap

def tableConverter():
    table_conv = TableConverter.make_stage(
    name='table_conv', 
    output_format='numpyDict',
    )
    return table_conv

In [31]:
col_remap = colRemapper(band_dict_err)
table_conv = tableConverter()

# Inform & Estimate

In [32]:
def informTrainZ():
    inf = TrainZInformer.make_stage(
    name = 'inform_TrainZ',
    model = 'trainz.pkl',
    hdf5_groupname=""
    )
    return inf

def estimateTrainZ(info):
    est = TrainZEstimator.make_stage(
    name = 'estimate_TrainZ',
    model = 'trainz.pkl', 
    hdf5_groupname=""
    )
    return est

In [33]:
def informCMNN():
    inf = Inform_CMNNPDF.make_stage(
    name = 'inform_CMNN',
    model = 'cmnn.pkl',
    hdf5_groupname=""
    )
    return inf

def estimateCMNN(info):
    est = CMNNPDF.make_stage(
    name = 'estimate_CMNN',
    model = 'cmnn.pkl', 
    hdf5_groupname=""
    )
    return est

In [34]:
def informGPz():
    inf = GPzInformer.make_stage(
    name = 'inform_GPz',
    model = 'gpz.pkl',
    hdf5_groupname=""
    )
    return inf

def estimateGPz(info):
    est = GPzEstimator.make_stage(
    name = 'estimate_GPz',
    model = 'gpz.pkl', 
    hdf5_groupname=""
    )
    return est

In [35]:
def informPZFlow():
    inf = PZFlowInformer.make_stage(
    name = 'inform_PZFlow',
    model = 'pzflow.pkl',
    hdf5_groupname=""
    )
    return inf

def estimatePZFlow(info):
    est = PZFlowEstimator.make_stage(
    name = 'estimate_PZFlow',
    model = 'pzflow.pkl', 
    hdf5_groupname=""
    )
    return est

In [36]:
def informFZBoost():
    info = FlexZBoostInformer.make_stage(
    name ='inform_FZBoost', 
    model ='fzboost.pkl', 
    hdf5_groupname='',
    )
    return info

def estimateFZBoost(info):
    est = FlexZBoostEstimator.make_stage(
    name='est_FZBoost', 
    nondetect_val=np.nan,
    model= info,
    hdf5_groupname='',
    aliases=dict(input='test_data', output='fzboost_estim'),
    nzbins = 100 
    )
    return est 

In [37]:
inf_est_dict = {'TrainZ': [informTrainZ, estimateTrainZ],
               'CMNN': [informCMNN, estimateCMNN], 
               'GPz': [informGPz, estimateGPz], 
               'PZFlow': [informPZFlow, estimatePZFlow], 
               'FZBoost': [informFZBoost, estimateFZBoost]}

In [38]:
# 'invz': invRedshift,

spec_dict = {'BOSS': specSelectBOSS, 
             'DEEP2': specSelectDEEP2, 
             'GAMA': specSelectGAMA,
             'HSC': specSelectHSC, 
             'VVDSf02': specSelectVVDSf02, 
             'zCOSMOS': specSelectzCOSMOS } 

inf_est_dict = {'TrainZ': [informTrainZ, estimateTrainZ],
               'CMNN': [informCMNN, estimateCMNN], 
               'GPz': [informGPz, estimateGPz], 
               'PZFlow': [informPZFlow, estimatePZFlow], 
               'FZBoost': [informFZBoost, estimateFZBoost] }

In [39]:
# import ceci 

# pr = ceci.Pipeline.read(path_lst_1[0])#parent_dir+directory+"/invz=0.33672517538070684_lsstErr_pzflow.yml")
# pr.run()

# ## 1) terminal: go to path up to invz_lsstErr_pzflow, then run these 2 lines 
# ## 2)  make list/txt file with list of paths to files made by big F

# ## do 1) 
# ## open virtual env
# ## python 
# ## import ceci 
# ## run the 2 lines of code above 


# ### at the end we can put this into a .py file that we can run at the command line 

# ## %cd ? 

In [40]:
## more config parameters/better config parameters
## have to give path above to estimator model instead of get_handle('model')
## fix truncated parameter printing in help(...)

# Big F's

In [41]:
# Make sure to change the first argument of testSet
# testData = testSet(ntest, seed2)

testData = testSet(100, 39)
testData.run()

test_data = DS.read_file("test_Data", TableHandle, "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/output_test_set.pq")

Inserting handle into data store.  model: /Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/data/trained_flow.pkl, test_set


Inserting handle into data store.  output_test_set: inprogress_output_test_set.pq, test_set


FileNotFoundError: 

In [ ]:
#lsstErr = lsstError(band_dict, seed3)

bands = ['u','g','r','i','z','y']
band_dict = {band: f"mag_{band}_lsst" for band in bands}
band_dict_err = {f'mag_{band}_lsst_err':f'mag_err_{band}_lsst' for band in bands}

lsstErr = lsstError(band_dict, 172)
lsstErr.connect_input(test_data) ## might be wrong; passing in a file not a stage 
lsstErr.run()

lsst_Err = DS.read_file("test_Data", TableHandle, "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/output_lsst_error.pq")

Inserting handle into data store.  input: None, lsst_error


KeyError: 'Unknown file format , supported types are{list(FILE_FORMAT_SUFFIXS.keys())}'

In [ ]:
## for inverse redshift incompleteness:

pivot_ls = [1.0, 1.4] 

## TrainZ

In [ ]:
def bigF0(degrader, name, pathname, ntrain, ntest, seed1, seed2, seed3, nbins):
    
    ##things you need
    #grid = makeGrid(0, 2.5, nbins) 
    bands = ['u','g','r','i','z','y']
    band_dict = {band: f"mag_{band}_lsst" for band in bands}
    band_dict_err = {f'mag_{band}_lsst_err':f'mag_err_{band}_lsst' for band in bands}

    # modelData = makeModel()
    
    trainData = trainSet(ntrain, seed1)
    deg = degrader(ntrain) 
    
    # testData = testSet(ntest, seed2)

    #lsstErr = lsstError(band_dict, seed3)
    infTrainZ = informTrainZ()
    estTrainZ = estimateTrainZ(infTrainZ)

    ##pipeline and yml
    pipe = ceci.Pipeline.interactive()
    stages = [
        trainData, 
        deg, 
        #testData, 
        #lsstErr,  
        infTrainZ, 
        estTrainZ]

    for stage in stages:
        pipe.add_stage(stage)

    deg.connect_input(trainData)
    lsstErr.connect_input(testData)

    infTrainZ.connect_input(deg) 
    estTrainZ.connect_input(infTrainZ, inputTag = 'model')
    estTrainZ.connect_input(lsstErr, inputTag = 'input') ## trucated out of docs :(

    pipe.initialize(
    dict(model=flow_file), dict(output_dir=".", log_dir=".", resume=False), None) 

    outpath = os.path.join(pathname, "% s_lsstErr_pzflow.yml" % name)
    pipe.save(outpath)
    return outpath 

In [ ]:
##run 

path_lst_0 = []
directory = "specSelection_lsstErr_TrainZ"
parent_dir = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/"
path_0 = os.path.join(parent_dir, directory)
os.makedirs(path_0, exist_ok=True)

In [ ]:
for key in spec_dict:
    path_lst_0.append(bigF0(spec_dict[key], key, path_0, 1000, 100, 17, 39, 172, 10))

In [ ]:
path_lst_0_invz = []
directory = "invz_lsstErr_TrainZ"
parent_dir = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/"
path_0_invz = os.path.join(parent_dir, directory)
os.makedirs(path_0_invz, exist_ok=True)

In [ ]:
for i in pivot_ls:
    path_lst_0_invz.append(bigF0(invRedshift(i), 'invz='+str(i), path_0_invz, 1000, 100, 17, 39, 172, 10))

## CMNN

In [ ]:
def bigF1(degrader, name, pathname, ntrain, ntest, seed1, seed2, seed3, nbins):
    
    ##things you need
    #grid = makeGrid(0, 2.5, nbins) 
    bands = ['u','g','r','i','z','y']
    band_dict = {band: f"mag_{band}_lsst" for band in bands}
    band_dict_err = {f'mag_{band}_lsst_err':f'mag_err_{band}_lsst' for band in bands}

    # modelData = makeModel()
    
    trainData = trainSet(ntrain, seed1)
    deg = degrader(ntrain) 
    
    # testData = testSet(ntest, seed2)

    #lsstErr = lsstError(band_dict, seed3)
    infCMNN = informCMNN()
    estCMNN = estimateCMNN(infCMNN)

    ##pipeline and yml
    pipe = ceci.Pipeline.interactive()
    stages = [
        trainData, 
        deg, 
        #testData, 
        #lsstErr,  
        infCMNN, 
        estCMNN]

    for stage in stages:
        pipe.add_stage(stage)

    deg.connect_input(trainData)
    lsstErr.connect_input(testData)

    infCMNN.connect_input(deg) 
    estCMNN.connect_input(infCMNN, inputTag = 'model')
    estCMNN.connect_input(lsstErr, inputTag = 'input') ## trucated out of docs :(

    pipe.initialize(
    dict(model=flow_file), dict(output_dir=".", log_dir=".", resume=False), None) 

    outpath = os.path.join(pathname, "% s_lsstErr_CMNN.yml" % name)
    pipe.save(outpath)
    return outpath 

In [ ]:
path_lst_1 = []
directory = "specSelection_lsstErr_CMNN"
parent_dir = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/"
path_1 = os.path.join(parent_dir, directory)
os.makedirs(path_1, exist_ok=True)

In [ ]:
for key in spec_dict:
    path_lst_1.append(bigF1(spec_dict[key], key, path_1, 1000, 100, 17, 39, 172, 10))

In [ ]:
path_lst_1_invz = []
directory = "invz_lsstErr_CMNN"
parent_dir = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/"
path_1_invz = os.path.join(parent_dir, directory)
os.makedirs(path_1_invz, exist_ok=True)

In [ ]:
for i in pivot_ls:
    path_lst_1_invz.append(bigF1(invRedshift(i), 'invz='+str(i), path_1_invz, 1000, 100, 17, 39, 172, 10))

## GPz

In [ ]:
def bigF2(degrader, name, pathname, ntrain, ntest, seed1, seed2, seed3, nbins):
    
    ##things you need
    #grid = makeGrid(0, 2.5, nbins) 
    bands = ['u','g','r','i','z','y']
    band_dict = {band: f"mag_{band}_lsst" for band in bands}
    band_dict_err = {f'mag_{band}_lsst_err':f'mag_err_{band}_lsst' for band in bands}

    # modelData = makeModel()
    
    trainData = trainSet(ntrain, seed1)
    deg = degrader(ntrain) 
    
    # testData = testSet(ntest, seed2)

    #lsstErr = lsstError(band_dict, seed3)
    infGPz = informGPz()
    estGPz = estimateGPz(infGPz)

    ##pipeline and yml
    pipe = ceci.Pipeline.interactive()
    stages = [
        trainData, 
        deg, 
        #testData, 
        #lsstErr,  
        infGPz, 
        estGPz]

    for stage in stages:
        pipe.add_stage(stage)

    deg.connect_input(trainData)
    lsstErr.connect_input(testData)

    infGPz.connect_input(deg) 
    estGPz.connect_input(infGPz, inputTag = 'model')
    estGPz.connect_input(lsstErr, inputTag = 'input') ## trucated out of docs :(

    pipe.initialize(
    dict(model=flow_file), dict(output_dir=".", log_dir=".", resume=False), None) 

    outpath = os.path.join(pathname, "% s_lsstErr_GPz.yml" % name)
    pipe.save(outpath)
    return outpath 

In [ ]:
path_lst_2 = []
directory = "specSelection_lsstErr_GPz"
parent_dir = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/"
path_2 = os.path.join(parent_dir, directory)
os.makedirs(path_2, exist_ok=True)

In [ ]:
for key in spec_dict:
    path_lst_1.append(bigF2(spec_dict[key], key, path_2, 1000, 100, 17, 39, 172, 10))

In [ ]:
path_lst_2_invz = []
directory = "invz_lsstErr_GPz"
parent_dir = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/"
path_2_invz = os.path.join(parent_dir, directory)
os.makedirs(path_2_invz, exist_ok=True)

In [ ]:
for i in pivot_ls:
    path_lst_2_invz.append(bigF2(invRedshift(i), 'invz='+str(i), path_1_invz, 1000, 100, 17, 39, 172, 10))

## PZFlow

In [ ]:
def bigF3(degrader, name, pathname, ntrain, ntest, seed1, seed2, seed3, nbins):
    
    ##things you need
    #grid = makeGrid(0, 2.5, nbins) 
    bands = ['u','g','r','i','z','y']
    band_dict = {band: f"mag_{band}_lsst" for band in bands}
    band_dict_err = {f'mag_{band}_lsst_err':f'mag_err_{band}_lsst' for band in bands}

    # modelData = makeModel()
    
    trainData = trainSet(ntrain, seed1)
    deg = degrader(ntrain) 
    
    # testData = testSet(ntest, seed2)

    #lsstErr = lsstError(band_dict, seed3)
    infPZFlow = informPZFlow()
    estPZFlow = estimatePZFlow(infPZFlow)

    ##pipeline and yml
    pipe = ceci.Pipeline.interactive()
    stages = [
        trainData, 
        deg, 
        #testData, 
        #lsstErr,  
        infPZFlow, 
        estPZFlow]

    for stage in stages:
        pipe.add_stage(stage)

    deg.connect_input(trainData)
    # lsstErr.connect_input(testData)

    infPZFlow.connect_input(deg) 
    estPZFlow.connect_input(infPZFlow, inputTag = 'model')
    # estPZFlow.connect_input(lsstErr, inputTag = 'input') ## trucated out of docs :( 
    estPZFlow.connect_input(lsst_Err, inputTag = 'input') ## might be wrong, passing in file instead of stage, need to debug w alex

    pipe.initialize(
    dict(model=flow_file), dict(output_dir=".", log_dir=".", resume=False), None) 

    outpath = os.path.join(pathname, "% s_lsstErr_PZFlow.yml" % name)
    pipe.save(outpath)
    return outpath 

In [ ]:
# help(rail.creation.degradation.spectroscopic_selections)

In [ ]:
##run 

path_lst_3 = []
directory = "specSelection_lsstErr_PZFlow"
parent_dir = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/"
path_3 = os.path.join(parent_dir, directory)
os.makedirs(path_3, exist_ok=True)

In [ ]:
for key in spec_dict:
    path_lst_3.append(bigF3(spec_dict[key], key, path_3, 1000, 100, 17, 39, 172, 10))

ValueError: 
Some required inputs to the pipeline could not be found,
(or possibly your pipeline is cyclic):

Stage lsst_error is missing input(s): output_test_set
Stage estimate_PZFlow is missing input(s): output_lsst_error


In [ ]:
path_lst_3_invz = []
directory = "invz_lsstErr_PZFlow"
parent_dir = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/"
path_3_invz = os.path.join(parent_dir, directory)
os.makedirs(path_3_invz, exist_ok=True)

In [ ]:
for i in pivot_ls:
    path_lst_3_invz.append(bigF3(invRedshift(i), 'invz='+str(i), path_1_invz, 1000, 100, 17, 39, 172, 10))

## FlexZBoost

In [ ]:
def bigF4(degrader, name, pathname, ntrain, ntest, seed1, seed2, seed3, nbins):
    
    ##things you need
    #grid = makeGrid(0, 2.5, nbins) 
    bands = ['u','g','r','i','z','y']
    band_dict = {band: f"mag_{band}_lsst" for band in bands}
    band_dict_err = {f'mag_{band}_lsst_err':f'mag_err_{band}_lsst' for band in bands}

    # modelData = makeModel()
    
    trainData = trainSet(ntrain, seed1)
    deg = degrader(ntrain) 
    
    # testData = testSet(ntest, seed2)

    #lsstErr = lsstError(band_dict, seed3)
    infFZBoost = informFZBoost()
    estFZBoost = estimateFZBoost(infFZBoost)

    ##pipeline and yml
    pipe = ceci.Pipeline.interactive()
    stages = [
        trainData, 
        deg, 
        #testData, 
        #lsstErr,  
        infFZBoost, 
        estFZBoost]

    for stage in stages:
        pipe.add_stage(stage)

    deg.connect_input(trainData)
    lsstErr.connect_input(testData)

    infFZBoost.connect_input(deg) 
    estFZBoost.connect_input(infFZBoost, inputTag = 'model')
    estFZBoost.connect_input(lsstErr, inputTag = 'input') ## trucated out of docs :(

    pipe.initialize(
    dict(model=flow_file), dict(output_dir=".", log_dir=".", resume=False), None) 

    outpath = os.path.join(pathname, "% s_lsstErr_FZBoost.yml" % name)
    pipe.save(outpath)
    return outpath 

In [ ]:
path_lst_4 = []
directory = "specSelection_lsstErr_FZBoost"
parent_dir = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/"
path_4 = os.path.join(parent_dir, directory)
os.makedirs(path_4, exist_ok=True)

In [ ]:
for key in spec_dict:
    path_lst_4.append(bigF4(spec_dict[key], key, path_4, 1000, 100, 17, 39, 172, 10))

In [ ]:
path_lst_4_invz = []
directory = "invz_lsstErr_FZBoost"
parent_dir = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/"
path_4_invz = os.path.join(parent_dir, directory)
os.makedirs(path_4_invz, exist_ok=True)

In [ ]:
for i in pivot_ls:
    path_lst_4_invz.append(bigF4(invRedshift(i), 'invz='+str(i), path_1_invz, 1000, 100, 17, 39, 172, 10))

## Running things

In [ ]:
pr = ceci.Pipeline.read(path_2+"/BOSS_lsstErr_pzflow.yml")
pr.run()



Executing test_set
Command is:
OMP_NUM_THREADS=1   python3 -m ceci rail.creation.engines.flowEngine.FlowCreator   --model=/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/data/trained_flow.pkl   --name=test_set   --config=/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_lsstErr_pzflow/BOSS_lsstErr_pzflow_config.yml   --output=./output_test_set.pq 
Output writing to ./test_set.out

Job test_set has completed successfully!

Executing lsst_error
Command is:
OMP_NUM_THREADS=1   python3 -m ceci rail.creation.degradation.lsst_error_model.LSSTErrorModel   --input=./output_test_set.pq   --name=lsst_error   --config=/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_lsstErr_pzflow/BOSS_lsstErr_pzflow_config.yml   --output=./output_lsst_error.pq 
Output writing to ./lsst_error.out

Job lsst_error has completed successfully!

Executing train_set
Command is:
OMP_NUM_THREADS=1   p

0

In [ ]:
help(os.makedirs)


Help on function makedirs in module os:

makedirs(name, mode=511, exist_ok=False)
    makedirs(name [, mode=0o777][, exist_ok=False])
    
    Super-mkdir; create a leaf directory and all intermediate ones.  Works like
    mkdir, except that any intermediate path segment (not just the rightmost)
    will be created if it does not exist. If the target directory already
    exists, raise an OSError if exist_ok is False. Otherwise no exception is
    raised.  This is recursive.



# Test

In [44]:
def bigF_TEST1(degrader, name, pathname, ntrain, ntest, seed1, seed2, seed3, nbins):
    
    ##things you need
    #grid = makeGrid(0, 2.5, nbins) 
    bands = ['u','g','r','i','z','y']
    band_dict = {band: f"mag_{band}_lsst" for band in bands}
    band_dict_err = {f'mag_{band}_lsst_err':f'mag_err_{band}_lsst' for band in bands}

    # modelData = makeModel()
    
    trainData = trainSet(ntrain, seed1)
    deg = degrader(ntrain) 
    
    testData = testSet(ntest, seed2)

    lsstErr = lsstError(band_dict, seed3)
    infPZFlow = informPZFlow()
    estPZFlow = estimatePZFlow(infPZFlow)

    ##pipeline and yml
    pipe = ceci.Pipeline.interactive()
    stages = [
        trainData, 
        deg, 
        testData, 
        lsstErr,  
        infPZFlow, 
        estPZFlow]  

    for stage in stages:
        pipe.add_stage(stage)

    deg.connect_input(trainData)
    lsstErr.connect_input(testData)

    infPZFlow.connect_input(deg) 
    estPZFlow.connect_input(infPZFlow, inputTag = 'model')
    estPZFlow.connect_input(lsstErr, inputTag = 'input') ## trucated out of docs :( 
    #estPZFlow.connect_input(lsst_Err, inputTag = 'input') ## might be wrong, passing in file instead of stage, need to debug w alex

    pipe.initialize(
    dict(model=flow_file), dict(output_dir=".", log_dir=".", resume=False), None) 

    outpath = os.path.join(pathname, "% s_TEST1.yml" % name)
    pipe.save(outpath)
    return outpath 

In [45]:

path_lst_TEST1 = []
directory = "specSelection_TEST1"
parent_dir = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/"
path_TEST1 = os.path.join(parent_dir, directory)
os.makedirs(path_TEST1, exist_ok=True)


for key in spec_dict:
    path_lst_TEST1.append(bigF_TEST1(spec_dict[key], key, path_TEST1, 1000, 1000, 17, 39, 172, 10))

Inserting handle into data store.  output_specselection_deep2: inprogress_output_specselection_deep2.pq, specselection_deep2
Inserting handle into data store.  output_specselection_gama: inprogress_output_specselection_gama.pq, specselection_gama
Inserting handle into data store.  output_specselection_HSC: inprogress_output_specselection_HSC.pq, specselection_HSC
Inserting handle into data store.  output_specselection_VVDSf02: inprogress_output_specselection_VVDSf02.pq, specselection_VVDSf02
Inserting handle into data store.  output_specselection_zCOSMOS: inprogress_output_specselection_zCOSMOS.pq, specselection_zCOSMOS


In [48]:
def bigF_TEST2(degrader, name, pathname, ntrain, ntest, seed1, seed2, seed3, nbins):
    
    ##things you need
    #grid = makeGrid(0, 2.5, nbins) 
    bands = ['u','g','r','i','z','y']
    band_dict = {band: f"mag_{band}_lsst" for band in bands}
    band_dict_err = {f'mag_{band}_lsst_err':f'mag_err_{band}_lsst' for band in bands}

    # modelData = makeModel()
    
    trainData = trainSet(ntrain, seed1)
    deg = degrader(ntrain) 
    
    testData = testSet(ntest, seed2)

    lsstErr = lsstError(band_dict, seed3)
    infPZFlow = informPZFlow()
    estPZFlow = estimatePZFlow(infPZFlow)

    ##pipeline and yml
    pipe = ceci.Pipeline.interactive()
    stages = [
        trainData, 
        deg, 
        testData, 
        lsstErr,  
        infPZFlow, 
        estPZFlow]


    for stage in stages:
        pipe.add_stage(stage)

    deg.connect_input(trainData)
    lsstErr.connect_input(testData)

    infPZFlow.connect_input(deg) 
    estPZFlow.connect_input(infPZFlow, inputTag = 'model')
    estPZFlow.connect_input(lsstErr, inputTag = 'input') ## trucated out of docs :( 
    #estPZFlow.connect_input(lsst_Err, inputTag = 'input') ## might be wrong, passing in file instead of stage, need to debug w alex

    pipe.initialize(
    dict(model=flow_file), dict(output_dir=".", log_dir=".", resume=False), None) 

    outpath = os.path.join(pathname, "% s_TEST2.yml" % name)
    pipe.save(outpath)
    return outpath 

In [49]:

path_lst_TEST2 = []
directory = "specSelection_TEST2"
parent_dir = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/"
path_TEST2 = os.path.join(parent_dir, directory)
os.makedirs(path_TEST2, exist_ok=True)


for key in spec_dict:
    path_lst_TEST2.append(bigF_TEST2(spec_dict[key], key, path_TEST2, 10000, 10000, 17, 39, 172, 10))

In [50]:
#________________________#

In [51]:
out_dir1 = "outputs"
out_parent_dir1 = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_TEST1"
path_outs1 = os.path.join(out_parent_dir1, out_dir1)
os.makedirs(path_outs1, exist_ok=True)


In [52]:

os.chdir("/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_TEST1/outputs")

pr = ceci.Pipeline.read(path_TEST1+"/BOSS_TEST1.yml")
pr.run()


Executing test_set
Command is:
OMP_NUM_THREADS=1   python3 -m ceci rail.creation.engines.flowEngine.FlowCreator   --model=/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/data/trained_flow.pkl   --name=test_set   --config=/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_TEST1/BOSS_TEST1_config.yml   --output=./output_test_set.pq 
Output writing to ./test_set.out

Job test_set has completed successfully!

Executing lsst_error
Command is:
OMP_NUM_THREADS=1   python3 -m ceci rail.creation.degradation.lsst_error_model.LSSTErrorModel   --input=./output_test_set.pq   --name=lsst_error   --config=/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_TEST1/BOSS_TEST1_config.yml   --output=./output_lsst_error.pq 
Output writing to ./lsst_error.out

Job lsst_error has completed successfully!

Executing train_set
Command is:
OMP_NUM_THREADS=1   python3 -m ceci rail.creation.engines

0

In [56]:
import pandas as pd
df1train = pd.read_parquet("/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_TEST1/outputs/output_train_set.pq")

df1train

,redshift,mag_u_lsst,mag_g_lsst,mag_r_lsst,mag_i_lsst,mag_z_lsst,mag_y_lsst
0,0.933041,27.515276,27.286690,26.769602,26.087349,25.628199,25.478405
1,0.595904,27.316408,26.487122,25.609987,25.022789,24.816481,24.619503
2,0.503885,26.787773,26.411472,25.777828,25.382620,25.257975,25.065731
3,2.149223,29.424986,28.336277,27.484585,27.122864,26.690264,26.177797
4,1.330744,26.478308,26.149168,25.454374,24.850704,24.430229,23.861637
...,...,...,...,...,...,...,...
995,2.016123,26.238529,26.198832,26.058969,26.133110,26.050972,25.689487
996,0.520875,27.604046,26.637758,25.569271,25.078390,24.902649,24.692707
997,2.172287,29.186840,28.469145,27.898277,27.719030,27.374804,26.885008
998,1.234789,29.393019,28.316401,26.837587,25.619205,24.774239,24.047237


In [53]:
import pandas as pd
df1 = pd.read_parquet("/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_TEST1/outputs/output_specselection_boss.pq")

df1

,redshift,mag_u_lsst,mag_g_lsst,mag_r_lsst,mag_i_lsst,mag_z_lsst,mag_y_lsst
470,0.678802,22.524094,22.298061,21.065384,20.072044,19.666212,19.417404
557,0.496280,22.490469,21.483559,20.207443,19.443588,19.127865,18.890064


In [57]:
import pandas as pd
df2train = pd.read_parquet("/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_TEST2/outputs/output_train_set.pq")

df2train

,redshift,mag_u_lsst,mag_g_lsst,mag_r_lsst,mag_i_lsst,mag_z_lsst,mag_y_lsst
0,1.342823,25.937267,25.592773,25.302280,24.184923,23.383533,22.579002
1,1.191652,28.780466,28.075939,27.267124,26.668451,25.890570,25.420044
2,1.751760,26.832052,26.458172,26.087759,25.595860,24.988527,24.666149
3,0.620976,22.530554,22.241764,21.103287,20.136244,19.746765,19.453854
4,0.793243,27.564707,26.932222,26.285490,25.502998,25.277523,25.163303
...,...,...,...,...,...,...,...
9995,1.981384,27.963957,27.702507,27.336897,27.156370,26.714178,26.362583
9996,0.744960,27.017117,26.490191,25.695547,24.839457,24.637497,24.520649
9997,1.101568,27.205582,26.924608,26.450403,26.079315,25.530327,25.305614
9998,0.928054,28.203192,27.457575,26.576130,25.856819,25.445122,25.309990


In [54]:
out_dir2 = "outputs"
out_parent_dir2 = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_TEST2"
path_outs2 = os.path.join(out_parent_dir2, out_dir2)
os.makedirs(path_outs2, exist_ok=True)



os.chdir("/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_TEST2/outputs")

pr = ceci.Pipeline.read(path_TEST2+"/BOSS_TEST2.yml")
pr.run()

import pandas as pd
df2 = pd.read_parquet("/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_TEST2/outputs/output_specselection_boss.pq")

df2


Executing test_set
Command is:
OMP_NUM_THREADS=1   python3 -m ceci rail.creation.engines.flowEngine.FlowCreator   --model=/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/data/trained_flow.pkl   --name=test_set   --config=/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_TEST2/BOSS_TEST2_config.yml   --output=./output_test_set.pq 
Output writing to ./test_set.out

Job test_set has completed successfully!

Executing lsst_error
Command is:
OMP_NUM_THREADS=1   python3 -m ceci rail.creation.degradation.lsst_error_model.LSSTErrorModel   --input=./output_test_set.pq   --name=lsst_error   --config=/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_TEST2/BOSS_TEST2_config.yml   --output=./output_lsst_error.pq 
Output writing to ./lsst_error.out

Job lsst_error has completed successfully!

Executing train_set
Command is:
OMP_NUM_THREADS=1   python3 -m ceci rail.creation.engines

,redshift,mag_u_lsst,mag_g_lsst,mag_r_lsst,mag_i_lsst,mag_z_lsst,mag_y_lsst
525,0.298815,22.866861,21.576294,20.371439,19.647549,19.384043,19.054184
3830,0.506937,23.405199,20.913086,19.358709,18.569523,18.225611,17.970161
4847,0.649493,22.256865,21.764664,20.519007,19.611025,19.282930,19.013233
5592,0.423514,23.458931,20.948090,19.118870,18.525711,18.173027,17.956638
5605,0.648363,22.301899,21.645138,20.663469,19.834093,19.535355,19.293495
5835,0.648992,25.440855,22.707363,20.903374,19.688295,19.192822,18.862762
7164,0.426807,23.751444,20.760584,19.133968,18.386021,18.017616,17.826719
8306,0.619865,23.423046,22.094854,20.491508,19.485312,18.963287,18.576736
